In [21]:
 !wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-10-02 16:33:06--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-10-02 16:33:07--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3edfae60241cdb2109b4204389.dl.dropboxusercontent.com/cd/0/inline/BXT0rSaj79r_lrVjcuX5lC6x8x_963b7aExxd1gvpvIl_imUZEEiPxswmLUKPEfMfAgTkfaUdMDUEV265MSHL3LMq0s8YU8dsdoSNMpg1ybRCLbuQXnaPbKQExnU1yVWMFWABamDdJOf0nXcBV2I58-p/file# [following]
--2021-10-02 16:33:07--  https://uc3edfae60241cdb2109b4204389.dl.dropboxusercontent.com/cd/0/inline/BXT0rSaj79r_lrVjcuX5lC6x8x_963b7aExxd1gvpvIl_imUZEE

In [22]:
!unzip -d"/content/movie-reviews-dataset.zip"

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files
  -j  junk paths (do not make directories)   -aa treat ALL files as text
  -U  use escapes for all non-ASCII Unicode  -UU ignore any Unicode fields
  -C  mat

In [23]:
!unzip -q "/content/movie-reviews-dataset.zip"

replace movie-reviews-dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace __MACOSX/movie-reviews-dataset/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace __MACOSX/movie-reviews-dataset/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [24]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense,RNN,SimpleRNNCell,Embedding,Dropout

In [26]:
def prepdata(dir):
  data = text_dataset_from_directory(dir)
  return data.map(lambda text,label:(regex_replace(text,'<br />',''),label),)

In [25]:
testdata = prepdata('/content/movie-reviews-dataset/test')
traindata = prepdata('/content/movie-reviews-dataset/train')

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [27]:
for text_batch,label_batch in traindata.take(1):
  print(text_batch.numpy()[2])
  print(label_batch.numpy()[2])

b"A good film with strong performances (especially the two leads). The film is about two American girls who are caught with 6 kilo's heroin on an airport in Thailand. They're both thrown in prison and one of them signs a confession. Bill Pullman plays the lawyer who tries to get them out. All they have to do is find a Nick Parks who put the narcotics in the bag of one of the two girls. So far for the story which isn't that original (it has many resemblances with the better Return to Paradise).The acting and Newton Thomas Sigel's beautiful photography make this film worth to watch. A 7 out of 10."
1


In [28]:
model = Sequential()

In [29]:
model.add(Input(shape=(1,),dtype="string"))

In [30]:
max_tokens = 1500
max_len = 80
vectorize_layer = TextVectorization(
    max_tokens = max_tokens,
    output_mode = "int",
    output_sequence_length = max_len,
)

In [31]:
train_texts = traindata.map(lambda text , label :text)
vectorize_layer.adapt(train_texts)
model.add(vectorize_layer)

In [32]:
model.add(Embedding(max_tokens+1,128))


rnn = RNN(SimpleRNNCell(64),return_sequences=False,return_state=False)
model.add(rnn)
model.add(Dense(64,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))

In [33]:
model.compile(loss="binary_crossentropy",optimizer = "adam")

In [34]:
model.fit(traindata,epochs=9)

Epoch 1/9
782/782 [==============================] - 28s 35ms/step - loss: 0.6860
Epoch 2/9
782/782 [==============================] - 27s 35ms/step - loss: 0.6486
Epoch 3/9
782/782 [==============================] - 27s 35ms/step - loss: 0.6572
Epoch 4/9
782/782 [==============================] - 27s 35ms/step - loss: 0.6676
Epoch 5/9
782/782 [==============================] - 27s 35ms/step - loss: 0.6505
Epoch 6/9
782/782 [==============================] - 27s 35ms/step - loss: 0.6540
Epoch 7/9
782/782 [==============================] - 27s 35ms/step - loss: 0.6180
Epoch 8/9
782/782 [==============================] - 27s 35ms/step - loss: 0.5721
Epoch 9/9
782/782 [==============================] - 27s 35ms/step - loss: 0.5505


In [35]:
model.evaluate(testdata)

782/782 [==============================] - 13s 16ms/step - loss: 0.6131


0.6130914092063904

In [40]:
testdata

<MapDataset shapes: ((None,), (None,)), types: (tf.string, tf.int32)>

In [43]:
model.evaluate(["this is a movie"])

1/1 [==============================] - 0s 30ms/step - loss: 0.0000e+00


0.0